In [ ]:
import numpy as np
from nilearn import image
from nilearn.image import get_data
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from arch.bootstrap import IIDBootstrap
from scipy.stats import kurtosis
import re
import time
import json

# Data

In [ ]:
import glob
import pandas as pd
path= '/dicom/'
label_path='with_category.csv'
indicative="000"
df_labels=pd.read_csv(label_path,sep=';')
df_labels['id1']=indicative+df_labels['id'].astype(str)

df_labels['path']=path+df_labels['id1']+'/'
df_labels.path[1]
#glob.glob(path + "/*")

In [ ]:
# skipping studies with previous surgeries or artifacts
skip = ['00020023096','00020029351','00030026189','00030037155','00030037273','00050009584','00050002822',
        '00050004047','00050004076','00050004345','00050004468','00050004755']
df_labels=df_labels[~df_labels.id1.isin(skip)]

In [ ]:
path_health=df_labels[df_labels.label==1].path
path_pathologic=df_labels[df_labels.label==0].path
nx=45;ny=109;nz=91
space = 'mask'
dat_healt=getDataFromDirectoryHPLE(path_health.values,space,nx,ny,nz)
dat_pathologic=getDataFromDirectoryHPLE(path_pathologic.values,space,nx,ny,nz)

In [ ]:
print('health neuroimaging: ',len(dat_healt))
print('pathologic neuroimaging: ',len(dat_pathologic))

# Entropy of local values permuted between hemispheres (HPLE)

In [ ]:
quantiles = [0.25,0.5,0.75]

In [ ]:
%%time
results_health = imageCrossPermutEntropy1(dat_healt,quantiles,2)

In [ ]:
results_health['mean_entropy_y'][0:10]

In [ ]:
results_health['mean_entropy_x'][0:10]

In [ ]:
%%time
results_pathol = imageCrossPermutEntropy1(dat_pathologic,quantiles,2)

In [ ]:
results_pathol['mean_entropy_y'][0:10]

In [ ]:
results_pathol['mean_entropy_x'][0:10]

In [ ]:
r2=np.divide(results_pathol['mean_entropy_x'],results_pathol['mean_entropy_y'])

In [ ]:
r1=np.divide(results_health['mean_entropy_x'],results_health['mean_entropy_y'])

In [ ]:
np.mean(r2)

In [ ]:
np.mean(r1)

In [ ]:
r1[0:10]

In [ ]:
r2[0:10]

In [ ]:
# Save healty
dumped_healty = json.dumps(results_health, cls=NumpyEncoder) # , sort_keys=False, indent=4

with open('imageCrossPermutEntropy_helthy_V2.json', 'w') as f:
    json.dump(dumped_healty, f) # , sort_keys=False, indent=4

In [ ]:
# Save patholigic
dumped_pathologic = json.dumps(results_pathol, cls=NumpyEncoder) # , sort_keys=False, indent=4

with open('imageCrossPermutEntropy_pathologic_V2.json', 'w') as f:
    json.dump(dumped_pathologic, f) # , sort_keys=False, indent=4

In [ ]:
# Load data healty
with open('imageCrossPermutEntropy_helthy_V2.json', 'r') as fp:
    helth = json.load(fp)
results_health = json.loads(helth) 

In [ ]:
# Load data pathologic
with open('imageCrossPermutEntropy_pathologic_V2.json', 'r') as fp:
    pathologic = json.load(fp)
results_pathol = json.loads(pathologic) 

# Confusion matrix according to significance levels

In [ ]:
p_ref=np.arange(0+0.01,0.2+0.01,0.01)
p_ref=np.sort(np.append(p_ref,0.001))
HPLE_perform = HPLE(results_health,results_pathol,p_ref,'t_test')

In [ ]:
results_pathol.keys()

In [ ]:
f, (ax) = plt.subplots(1, 1,figsize=(10, 5))
t1 = np.arange(0.0, 1.0, 0.01)
for v in ['Acc','Precision','Recall','F1_score']:
    plt.plot(p_ref, HPLE_perform[v], label=v)

ax.set(xlabel='Significance level '+r'$\alpha$',ylabel= 'Indicator values')
plt.legend(bbox_to_anchor=(.4, 1), ncol=2, title='Indicator')
plt.show()